In [ ]:
%pip install relationalai networkx matplotlib

In [1]:
import relationalai as rai
from relationalai.std import aggregates,strings,dates

# Create a model named "tokenflow_test1".
model = rai.Model("tokenflow_test1")

Connected to debugger at ws://0.0.0.0:8080/ws/program...


In [3]:
# define entities for SF input tables; we will use it to populate the ontology entities
Virtuals_agents = model.Type("Virtuals_agents", source="tokenflow.public.virtuals_agents")
Token_snapshot_raw = model.Type("Token_snapshot_raw", source="tokenflow.public.token_snapshot")
Running_token_balances = model.Type("Running_token_balances", source="tokenflow.public.running_token_balances")

In [4]:
# print the columns in the SF input tables
print(Virtuals_agents.known_properties())
print(Token_snapshot_raw.known_properties())
print(Running_token_balances.known_properties())

['snowflake_id', 'agent_token_address', 'name', 'symbol', 'description', 'created_at', 'lp', 'tba', 'wallets', 'is_public', 'is_premium', 'x_username', 'status', 'extra_data', 'added_at']
['snowflake_id', 'name', 'token_address', 'snapshot_time', 'transfer_count', 'transfer_amount', 'buy_count', 'buy_amount', 'sell_count', 'sell_amount', 'mint_count', 'mint_amount', 'burn_count', 'burn_amount', 'total_supply', 'holder_count', 'usd_price', 'tvl']
['snowflake_id', 'token_address', 'holder', 'balance_change_time', 'running_balance']


## Ontology

In [5]:
# define a Token entity, where token_address is the unique identifier
# there are two sources to get Token
# first, we create a Token instance for every token_address found in tokenflow.public.token_snapshot table
Token = model.Type("Token")
with model.rule():
    tsr = Token_snapshot_raw()
    t = Token.add(token_address=tsr.token_address)
    t.set(name=tsr.name)
    t.set(lowercase_name=strings.lowercase(tsr.name))

In [6]:
# the second source for Token instance is from tokenflow.public.running_token_balances table
# in this case we need to lookup the name of the token_address from tokenflow.public.virtuals_agents
with model.rule():
    r = Running_token_balances()
    v = Virtuals_agents(agent_token_address=r.token_address)
    t = Token.add(token_address=r.token_address)
    t.set(name=v.name)
    t.set(lowercase_name=strings.lowercase(v.name))

In [7]:
# query a few instances of Token entity
with model.query() as select:
    v = Token()
    aggregates.top(10, v)
    z = select(v.name, v.lowercase_name, v.token_address)
print(z.results)

                     name          lowercase_name  \
0          $TRUST ME BROs          $trust me bros   
1                   Azara                   azara   
2               Baldashin               baldashin   
3               Bizonacci               bizonacci   
4              BuddhaJung              buddhajung   
5              HyperWaifu              hyperwaifu   
6  Metropolis by Virtuals  metropolis by virtuals   
7                 NODERZZ                 noderzz   
8           Puppet Master           puppet master   
9               Vainguard               vainguard   

                                token_address  
0  0xc841b4ead3f70be99472ffdb88e5c3c7af6a481a  
1  0x3e1f498f9ad7c505e33bf9080cc93dcaf057d29f  
2  0x7e74e0e4d58b3b5ac68af071bbd411f554e0a516  
3  0xb4f6e4455dce4557fd74055a4b4584c4b34968aa  
4  0x1db3def03b0fe4a602bf1acdf66916aa778b46a4  
5  0xdaf40742384cff4721b259bddbf23b52a3715618  
6  0x73cde00aeb52ed16a74c6e9d9e22514999c91d80  
7  0x2acd6a246157bf51636d06a8320

In [8]:
# define Token_snapshot entity, it has two unique identifiers, token and snapshot_time
Token_snapshot = model.Type("Token_snapshot")
with model.rule():
    tsr = Token_snapshot_raw()
    t = Token(token_address=tsr.token_address)
    ts = Token_snapshot.add(token=t, snapshot_time=tsr.snapshot_time)
    
    ts.set(holder_count=tsr.holder_count, usd_price=tsr.usd_price, tvl=tsr.tvl)

In [9]:
# query a few instances of Token_snapshot entity
with model.query() as select:
    v = Token_snapshot()
    aggregates.top(10, v)
    z = select(v.token.name, v.token.token_address, v.snapshot_time, v.holder_count)

print(z.results)

               name                               token_address  \
0            0xMonk  0x06abb84958029468574b28b6e7792a770ccaa2f6   
1             0xRay  0xac532562e3b31151a933b11c8d4387cb1ec61c70   
2   Agent YieldDefi  0x812f21e579da8121723c4893b38966c56f7795dd   
3      BaseHoundBot  0xccdf2cbabfa37878125ab2d20bfcb9328b7ab3cf   
4        Eye Future  0x5537a24ad7e8d68aec165dcff6d2f8c23605417f   
5  Michael De Santa  0x2cf29b1e910cc0d45d8151b84d4dfde1788cf30e   
6    NA by virtuals  0xa2935744de0056231e52ca37a8744c2fb532ed5e   
7         Stellaris  0xf24e6fdc85e2ad23884a47b908ef38befe48d01c   
8        Tradescoop  0xf404bc113f4fc7c2447cb2556dcf5a56e29fa2dd   
9         Vainguard  0xe709c929f04044310f30490ab42812270bf299b4   

        snapshot_time  holder_count  
0 2025-04-01 08:00:00         20643  
1 2025-04-01 08:00:00            28  
2 2025-04-01 08:00:00           611  
3 2025-04-01 08:00:00          1168  
4 2025-04-01 08:00:00         21960  
5 2025-04-01 08:00:00           14

In [10]:
# add a property of Token entity to keep track of the latest snapshot of each token
with model.rule():
    ts = Token_snapshot()
    aggregates.bottom(1, ts.snapshot_time, ts, per=[ts.token])
    ts.token.set(latest=ts)

In [11]:
# define Balance entity to track the running balance for each token and each holder at a given blance_change_time
Balance = model.Type("Balance")
with model.rule():
    r = Running_token_balances()
    t = Token(token_address=r.token_address)
    Balance.add(token=t, holder=r.holder, balance_change_time=r.balance_change_time).set(running_balance=r.running_balance)

In [12]:
# define a derived entity from Balance to track the latest Balance for each token and each holder
Balance_latest = model.Type("Balance_latest")
with model.rule():
    b = Balance()
    aggregates.bottom(1, b.balance_change_time, b, per=[b.token, b.holder])
    b.set(Balance_latest)

In [ ]:
# define a derived entity from Balance to track the Balance for each token and each holder 3 month ago
Balance_3month_ago = model.Type("Balance_3month_ago")
with model.rule():
    b = Balance()
    time_now = aggregates.max(b.balance_change_time)
    b.balance_change_time <= time_now - dates.months(3)
    aggregates.bottom(1, b.balance_change_time, b, per=[b.token, b.holder])
    b.set(Balance_3month_ago)

In [13]:
# query a few instances of Balance_latest instance 
with model.query() as select:
    b = Balance_latest()
    aggregates.top(10, b)
    z = select(b.token.name, b.holder, b.balance_change_time, b.running_balance)
print(z.results)

             name                                      holder  \
0       AI ROCKET  0xf5aca5c3a0b70f847de4652ac77bd601ccfe8339   
1      AIVeronica  0x0a2854fbbd9b3ef66f17d47284e7f899b9509330   
2         Acolyte  0xf8c7e892fdab0dd3ed047b9eb42e6ed10a158e0f   
3       EchoLeaks  0x1895a6b620d71dd351a6635a64beaa24636c929f   
4       Gigabrain  0xae884e910a284e5e022c2b63db610e2f50d46f4f   
5      Polytrader  0x382ffce2287252f930e1c8dc9328dac5bf282ba1   
6            Rekt  0xb51d276418fcf23733877fb0629617667c78c831   
7         Sympson  0xf22482a194b7df297599a8d05c418c4f36cbe560   
8  WAI Combinator  0xc7d3ab410d49b664d03fe5b1038852ac852b1b29   
9          sekoia  0x47686106181b3cefe4eaf94c4c10b48ac750370b   

  balance_change_time          running_balance  
0 2025-04-01 08:00:00      16256024.2629466150  
1 2025-04-01 08:00:00                   -1E-10  
2 2025-04-01 08:00:00          3646.0191794016  
3 2025-04-01 08:00:00          -147.4668860267  
4 2025-04-01 08:00:00        138194.706

## Sample queries

#### "How many holders of AZARA are there?"
```
select holder_count from token_snapshot 
where lower(name)='azara'
order by snapshot time desc
limit 1

In [19]:
# "How many holders of AZARA are there?"
with model.query() as select:
    v = Token()
    v.lowercase_name == 'azara'
    z = select(v.name, v.token_address, v.latest.snapshot_time, v.latest.holder_count)
print(z.results)

    name                               token_address       snapshot_time  \
0  Azara  0x3e1f498f9ad7c505e33bf9080cc93dcaf057d29f 2025-04-01 08:00:00   

   holder_count  
0            30  


#### "How many holders currently have more than 20,000 REKT?"
```
with balances as (
    select holder,
    max_by(running_balance, balance_change_time) as balance
    from running_token_balances b
    left join virtuals_agents a on b.token_address=a.agent_token_address
    where lower(a.name)='aixbt'
    group by holder
)
select count(*) from balances
where balance>20000

In [20]:
#"How many holders currently have more than 20,000 REKT?"
with model.query() as select:
    b = Balance_latest()
    b.token.lowercase_name == "rekt"
    b.running_balance > 20000
    c = aggregates.count(b, b.holder, per=[b.token])
    z = select(b.token.name, c)
print(z.results)

   name  result
0  Rekt       1


#### How many holders have tokens from 5 or more Virtuals agents?
```
with balances as (
    select holder,
    token_address,
    max_by(running_balance, balance_change_time) as balance
    from running_token_balances
    group by holder, token_address
    having balance > 0
), 
holders_with_5 as (
    select holder,
    count(*) as token_count
    from balances
    group by holder
    having token_count >= 5
)
select count(*) from holders_with_5

In [15]:
#How many holders have tokens from 5 or more Virtuals agents?
with model.query() as select:
    b = Balance_latest()
    b.running_balance > 0
    num_token = aggregates.count(b, b.token, per=[b.holder])
    num_token > 1
    num_holder = aggregates.count(b.holder)
    z = select(num_holder)
print(z.results)

   result
0       7


#### How many new token holders that didn't own before but now do are there for REKT today compared to 3 months ago?
```
"with holders_3m as (
    select holder,
    max_by(running_balance, balance_change_time) as balance
    from running_token_balances b
    left join virtuals_agents a
    on b.token_address=a.agent_token_address
    where lower(a.name)='aixbt'
    and balance_change_time <= dateadd('month', -3, sysdate())
    group by holder
    having balance > 0
), 
holders_now as (
    select holder,
    max_by(running_balance, balance_change_time) as balance
    from running_token_balances b
    left join virtuals_agents a
    on b.token_address=a.agent_token_address
    where a.name='aixbt'
    group by holder
    having balance > 0
)
select count(*) from holders_now 
where holder not in (select holder from holders_3m)

In [18]:
# How many new token holders that didn't own before but now do are there for REKT today compared to 3 months ago?,
with model.query() as select:
    b = Balance_latest()
    b.token.lowercase_name == "rekt"
    b.running_balance > 0
    with model.not_found():
        b2 = Balance_3month_ago()
        b2.token.lowercase_name == "rekt"
        b2.holder == b.holder
    num_new_holder = aggregates.count(b.holder)
    z = select(num_new_holder)

print(z.results)

   result
0       1
